In [5]:
pip install segmentation-models-pytorch


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.2 MB 6.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.8/2.2 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16460 sha256=8767e58249f2c420dffd3077e390cb2ea387c779a717d4d0fa71995447c11d4c
  Stored in directory: c:\users\deeks_w4ub1k8\appdata\local\pip\cache\wheels\9c\3f\43\e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=61016 sha256=a12516b541382f3bb72db20ff78cc86b275079625b8119bef24c775a4483963f
  Stored in di

In [26]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import segmentation_models_pytorch as smp
import torch.optim as optim
from tqdm import tqdm

# Step 1: Check if a GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 2: Custom Dataset for Cityscapes
class CityscapesDataset(Dataset):
    def __init__(self, left_img_dir, gt_dir, transform=None):
        self.left_img_dir = left_img_dir
        self.gt_dir = gt_dir
        self.transform = transform
        
        # Collect all image paths from the leftImg8bit directory
        self.images = []
        for city in os.listdir(left_img_dir):
            city_path = os.path.join(left_img_dir, city)  # Access each city folder directly
            if os.path.isdir(city_path):
                for filename in os.listdir(city_path):
                    if filename.endswith('.png') or filename.endswith('.jpg'):
                        self.images.append(os.path.join(city_path, filename))
        
        print(f"Found {len(self.images)} images in {self.left_img_dir}")  # Debug statement

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        image = Image.open(img_name).convert("RGB")
        
        # Extract the city name from the image path
        city_name = img_name.split(os.sep)[-2]  # Get the city name from the image path
        
        # Construct the corresponding label path in the gtFine directory
        base_filename = os.path.basename(img_name).replace('leftImg8bit', 'gtFine_labelIds')  # Adjust the filename here
        label_name = os.path.join(self.gt_dir, 'train', city_name, base_filename)

        # Open the label image
        label = Image.open(label_name)

        if self.transform:
            image = self.transform(image)
            label = self.transform(label)

        return image, label

# Step 3: Define your dataset transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

# Step 4: Load the dataset
left_img_dir = 'C:\\LeftImg\\train'  # Path to left images
gt_dir = 'C:\\gtFine'  # Path to ground truth masks
train_dataset = CityscapesDataset(left_img_dir, gt_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)  # Adjusted batch size

# Step 5: Initialize the DeepLabV3+ model
model = smp.DeepLabV3Plus('resnet34', in_channels=3, classes=34, activation=None).to(device)

# Step 6: Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Step 7: Set training parameters
num_epochs = 5  # Adjust as needed

# Step 8: Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in tqdm(train_loader):
        if images is None or labels is None:  # Skip if data loading failed
            continue
        
        images, labels = images.to(device), labels.to(device)  # Move data to device
        labels = labels.long()  # Ensure labels are LongTensor

        # Print the shape of labels before any processing
        print(f"Original labels shape: {labels.shape}")  # Debugging output

        # Check if the labels need to be converted to class indices
        if labels.dim() == 4:  # If labels have 4 dimensions
            # Assuming the third dimension is channels (e.g., RGB) and we want to convert to class indices
            labels = labels.argmax(dim=1)  # Convert one-hot or multi-channel to single-channel class indices
            # Now, labels shape should be (N, H, W)

        # Print the shape of labels after processing
        print(f"Processed labels shape: {labels.shape}")  # Debugging output

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Check the shape of the outputs
        print(f"Outputs shape: {outputs.shape}")  # Debugging output

        # Calculate loss
        loss = criterion(outputs, labels)  # Use outputs directly

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print loss for the epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training completed.")




Found 2975 images in C:\LeftImg\train


  0%|                                                                                          | 0/744 [00:00<?, ?it/s]

Original labels shape: torch.Size([4, 3, 512, 512])
Processed labels shape: torch.Size([4, 512, 512])
Outputs shape: torch.Size([4, 34, 512, 512])


  0%|                                                                                | 1/744 [00:06<1:15:42,  6.11s/it]

Original labels shape: torch.Size([4, 3, 512, 512])
Processed labels shape: torch.Size([4, 512, 512])
Outputs shape: torch.Size([4, 34, 512, 512])


...

Original labels shape: torch.Size([4, 3, 512, 512])
Processed labels shape: torch.Size([4, 512, 512])
Outputs shape: torch.Size([4, 34, 512, 512])


100%|█████████████████████████████████████████████████████████████████████████████▉| 743/744 [1:58:44<00:09,  9.39s/it]

Original labels shape: torch.Size([3, 3, 512, 512])
Processed labels shape: torch.Size([3, 512, 512])
Outputs shape: torch.Size([3, 34, 512, 512])


100%|██████████████████████████████████████████████████████████████████████████████| 744/744 [1:58:51<00:00,  9.59s/it]

Epoch [5/5], Loss: 0.0013
Training completed.


In [28]:
# After the training loop
torch.save(model.state_dict(), "deeplabv3plus_cityscapes.pth")
print("Model saved successfully.")


Model saved successfully.
